In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
model_name: str = "../sk-trinity-with-sum/models/model-00epoch-8750steps-1.0742loss"
tokenizer_name: str = "../models/sk-trinity-6250step"
max_length: int = 512
device: str = "cuda:0"
skt: bool = True

In [11]:
tokenizer_kwargs = {"padding_side": "left"}
if skt:
    tokenizer_kwargs.update(
        bos_token="</s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
    )

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, **tokenizer_kwargs)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [12]:
def datum_to_string(datum):
    text = f"제목: {datum['title']}\n"

    summarizations = datum.get("summarizations")
    if summarizations:
        summarization = " ".join(summarizations).replace("\n", " ")
        text = f"요약: {summarization}\n" + text
    return text


input_examples = [
    {"title": "거짓말에는 종점이 있다", "summarizations": []},
    {"title": "나에게 자동차란?", "summarizations": []},
    {"title": "7. 모기", "summarizations": []},
    {"title": "[육아 에세이] 아이와의 사랑과 전쟁", "summarizations": []},
    {"title": "노랑 사랑", "summarizations": []},
    {"title": "창문 넘어 도망친 소설들", "summarizations": [
      "독서량과 학습 결과를 비교하기 위해 독서량과 학습 결과를 비교하기 위해 설문조사를 한 결과, 독서량과 학습 결과를 비교하기 위해 '5번 기타'에 체크한 수치가 1000권쯤이었다.",
      "문예창작을 전공하기 전까지만 해도 소설을 그리 즐겨 읽는 편은 아니었지만, 본격적으로 글을 배우게 되면서 소설의 맛을 알게 되었다.",
      "<그들에게 린디합을>, <우연의 신>, <작은 동네>, <우아한 밤과 고양이들> 등 각 작품들에 담긴 이야기에 푹 빠져 시간을 보내다 보면 마치 잠시 다른 세계에 다녀온 것 같은 기분이 들기도 한다.",
      "문학 이외에 육하원칙이 지배하는 신문기사와 주관적인 내면고백 덩어리로 보이는 문학이 실제 인간이 저지르는 일들을 더 잘 설명해줄 때가 많다."
    ]},
    {"title": "나지막이 내뱉은 말 '끝'", "summarizations": [
      "올림픽 2연패 양궁 국가대표 오진혁 선수가 남자 단체 결승전에서 마지막 화살을 쏜 직후 내뱉은 말이 화제다.",
      "지금까지 살아오면서 운에 기대본 적도 없지만, 스스로에게 확신해본 적도 없는 것 같다."
    ]},
    {"title": "깊은 호수(2)", "summarizations": [
      "깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진"
    ]},
    {"title": "존재 앞의 겸손", "summarizations": [
      "전역 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사..............................."
    ]},
    {"title": "전역 후에 헬스를 시작한 방법", "summarizations": [
      "군대에서 배운 딥스, 턱걸이 그리고 가슴 운동은 스쿼트를 기초로 시작했지만 초보자가 운동을 지속한 방법과, 과잠 입은 선후배들이 갑옷 대신 과잠 입은 선후배들이다.",
      "군대에서 이후 미디어 속 인물들의 몸짱이 비현실적이라는 생각을 버리고 운동의 빈도가 전보다 높아졌으며 운동의 빈도가 전보다 높아졌다."
    ]},
]

prompts = [datum_to_string(example) for example in input_examples]

In [13]:
prompts

['제목: 거짓말에는 종점이 있다\n',
 '제목: 나에게 자동차란?\n',
 '제목: 7. 모기\n',
 '제목: [육아 에세이] 아이와의 사랑과 전쟁\n',
 '제목: 노랑 사랑\n',
 "요약: 독서량과 학습 결과를 비교하기 위해 독서량과 학습 결과를 비교하기 위해 설문조사를 한 결과, 독서량과 학습 결과를 비교하기 위해 '5번 기타'에 체크한 수치가 1000권쯤이었다. 문예창작을 전공하기 전까지만 해도 소설을 그리 즐겨 읽는 편은 아니었지만, 본격적으로 글을 배우게 되면서 소설의 맛을 알게 되었다. <그들에게 린디합을>, <우연의 신>, <작은 동네>, <우아한 밤과 고양이들> 등 각 작품들에 담긴 이야기에 푹 빠져 시간을 보내다 보면 마치 잠시 다른 세계에 다녀온 것 같은 기분이 들기도 한다. 문학 이외에 육하원칙이 지배하는 신문기사와 주관적인 내면고백 덩어리로 보이는 문학이 실제 인간이 저지르는 일들을 더 잘 설명해줄 때가 많다.\n제목: 창문 넘어 도망친 소설들\n",
 "요약: 올림픽 2연패 양궁 국가대표 오진혁 선수가 남자 단체 결승전에서 마지막 화살을 쏜 직후 내뱉은 말이 화제다. 지금까지 살아오면서 운에 기대본 적도 없지만, 스스로에게 확신해본 적도 없는 것 같다.\n제목: 나지막이 내뱉은 말 '끝'\n",
 '요약: 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진 사람이 되기를 바라는 사람은 어떤 모습인가, 깊은 호수 같은 마음을 가진\n제목: 깊은 호수(2)\n',
 '요약: 전역 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사 봉사...............................\n제목: 존재 앞의 겸손\n',
 '요약: 군대에서 배운 딥스, 턱걸이 그리고 가슴 운동은 스쿼트를 기초로 시작했지만 초보자가 운동을 지속한 방법과,

In [14]:
input_ids = tokenizer(
        prompts,
        add_special_tokens=True,
        padding="longest",
        return_tensors="pt",
        return_token_type_ids=False,
        return_attention_mask=False,
)["input_ids"].to(device)

In [23]:
outputs = model.generate(input_ids, 
    max_length, 
    do_sample=True,
    # repetition_penalty=2.0, 
    num_return_sequences=1,
    # num_beams=10,
    pad_token_id=tokenizer.pad_token_id
)

In [24]:
texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for text in texts:
    print(text)
    print()

제목: 거짓말에는 종점이 있다






가장 기억에 남는 거짓말은 아빠가 오늘 술 마셨어. 네가 오늘 시험 보는 줄 알어. 어제 내가 봤는데 네가 100점 맞은 줄 알았어. 등 
어떤 거짓말이었을까?
생각해보면 아마도 너는 항상 나한테 나쁜 말만 하잖아. 하는 식의, 
어릴 적 들었던 그런 말이 아니었을까 싶다.
내가 그때 했던 말이 그건 그렇고, 저건 별로다. 같은 말이었을까?


그렇게 어린 시절에는 
아무리 친한 친구들이라도 
모르는 사람한테는 정말 
상대방으로 하여금 불쾌하게 하고 싶지 않다
생각했었다. 
그러다가 내가 어른이 되었을 무렵 
상대의 잘못을 지적하게 되어도 
그래도 좀 마음이 편하긴 하더라. 
그럴 때면 나는 
마음의 부담을 벗은 듯 
그때의 나처럼 
조용히 고개를 끄덕이곤 했다.
하지만 
그런 일이 나에게도 
지금도 있다. 

사실 그때의 나는 
대화를 할 때 
상대의 이야기를 잘 듣지 못하고 
상대방의 입장을 공감하지 못했다. 
내 말은 상대방의 행동을 두고 
내가 한 말이 아니라 
아니,  왜 저런데?라고 생각했던 거지. 
그러나 지금은 
상대의 행동에 동의하지 못하는 게 
얼마 전에 겪은 내 과거의 일로부터 
이리저리 휘젓고 다니던 나에서 비롯된 건지  
아마 내가 어려서 했던 행동이 
이렇게 변했기 때문이 아닌가 싶다. 


 
, 

. 

, 

. 

. 

. 

. 


.

제목: 나에게 자동차란?



최근, 어느 온라인 커뮤니티에 올린 이야기가 생각났다. 
[어르신, 자전거를 타신다구요] 


얼마 전, 인터넷에서 아주 오래전 이야기를 끄집어냈다. 
[자전거가 정말 좋아서요. 저도 배우려고요 
다음날 아침, 제가 출근을 하기 위해 운전 중이었는데, 창밖을 보니 아파트 입구에 자전거가 보였다고...
아니, 어쩌면 아침마다 매일 자전거를 탔으니 매일 그 모습을 보기 위해 이 동네에 살고 계셨을지도 몰랐다. 






생각해보니, 나의 첫 시승도 몇 달 전, 자전거를 탄지 얼마 안 되어 벌어진 일이었다. 

어르신이라